# Crypto-PAn(Cryptography-based  Prefix-preserving Anonymization) アルゴリズムの理解


prefix preserving な　IPアドレス匿名化手法

参考：
- http://www.cc.gatech.edu/computing/Networking/projects/cryptopan/
- Jinliang Fan, Jun Xu, Mostafa H. Ammar, Sue B. Moon, Prefix-preserving IP address anonymization: measurement-based security evaluation and a new cryptography-based scheme, Computer Networks, Volume 46, Issue 2, 7 October 2004, Pages 253-272, ISSN 1389-1286, http://dx.doi.org/10.1016/j.comnet.2004.03.033.
- http://www.cc.gatech.edu/computing/Networking/projects/cryptopan/Crypto-PAn.1.0.tar.gz
- https://github.com/keiichishima/yacryptopan



In [39]:
import Crypto.Cipher.AES
import socket
import struct
from array import array
import ipaddress
import numpy as np

#Orig IP: 192.168.0.1
#Ano IP: 2.149.253.242
  

class CryptoPAn:
    def __init__(self,key):
        """ コンストラクタ
        :param key: 暗号化用バイト配列（３２バイト）（上位１６バイトは暗号鍵，下位１６バイトはパディング用）
        :type key: array 
        """
        self._init(key)

    def _init(self,key):
        self._cipher = Crypto.Cipher.AES.new(key[:16],Crypto.Cipher.AES.MODE_ECB)
        padding = array('B')
        padding.fromstring(self._cipher.encrypt(key[16:]))
        self._padding_int = self._to_int(padding)
        
    def changeKey(slef,key):
        """暗号化鍵を変更する
        """
        self._init(key)
    
    def MSB(self,byte_array):
        """最上位ビットを取得する
        """
        n = byte_array[0] >> 7
        return n
    
    def LSB(self,byte_array):
        """最下位ビットを取得する
        """
        n = self._to_int(byte_array)
        return n & 1

    def anonymize(self,orig_addr):
        """ IPアドレスを匿名化する
        :param orig_addr: IPアドレスの文字列
        :type orig_addr: string
        :return anonymizedIP: int
        :rtype anonymizedIP: int
        """
        # orig_addr を unicode にする
        if not isinstance(orig_addr,unicode):
            orig_addr = unicode(orig_addr)
            
        ip=ipaddress.ip_address(orig_addr)
        addr = int(ip)
        if ip.version == 4:
            N = 32
            ext_addr = addr << 96
        elif ip.version == 6:
            N = 128
            ext_addr = addr 
         
        flip_array = []
        
        for pos in range(N):
            prefix =  ext_addr >> (128-pos) << (128-pos) 
            padded_addr = prefix | (self._padding_int & (2**128-1 >> pos))
            self._input = self.__to_byte_array(padded_addr,16)
            output = array('B')
            output.fromstring(self._cipher.encrypt(self._input))
            # 論文だとLSBだけど実装はMSB
            flip_array.append(self.MSB(output))
        result = reduce(lambda x, y: (x << 1) | y, flip_array)
        
        anonymizedIP = addr ^ result
        if (ip.version == 4):
            anonymizedIP = ipaddress.IPv4Address(anonymizedIP)
        elif (ip.version  ==6):
            anonymizedIP = ipaddress.IPv6Address(anonymizedIP) 
        return anonymizedIP
    
    def _to_int(self,byte_array):
        """ byte配列をint型に変換する
        """
        return reduce(lambda x,y: x << 8 | y,byte_array) 
    
    def __to_byte_array(self,n,byte_cnt):
        """ int型をbyte配列に変換する
        """
        byte_array = array('B')
        for i in range(byte_cnt):
            byte_array.insert(0, (n >> (i * 8)) & 0xff)
        return byte_array
    
if __name__ == '__main__' :
    # 192.0.2.1 > 2.90.93.17
    # 2001:db8::1 > dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
    key = array('B',range(32))
    obj = CryptoPAn(key)
    ip = '192.0.2.1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.17):",anonymizedIP 
    ip = '192.0.2.2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.19):",anonymizedIP 
    ip = '2001:db8::1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00):",anonymizedIP 
    ip = '2001:db8::2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02):",anonymizedIP   

        

Orig IP: 192.0.2.1
Ano IP(2.90.93.17): 2.90.93.17
Orig IP: 192.0.2.2
Ano IP(2.90.93.19): 2.90.93.19
Orig IP: 2001:db8::1
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
Orig IP: 2001:db8::2
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02


In [40]:
import Crypto.Cipher.AES
import socket
import struct
from array import array
import ipaddress
import numpy as np

#Orig IP: 192.168.0.1
#Ano IP: 2.149.253.242
  

class CryptoPAn:
    def __init__(self,key):
        """ コンストラクタ
        :param key: 暗号化用バイト配列（３２バイト）（上位１６バイトは暗号鍵，下位１６バイトはパディング用）
        :type key: array 
        """
        self._init(key)

    def _init(self,key):
        self._cipher = Crypto.Cipher.AES.new(key[:16],Crypto.Cipher.AES.MODE_ECB)
        padding = array('B')
        padding.fromstring(self._cipher.encrypt(key[16:]))
        self._padding_int = self._to_int(padding)
        
    def changeKey(slef,key):
        """暗号化鍵を変更する
        """
        self._init(key)
    
    def MSB(self,byte_array):
        """最上位ビットを取得する
        """
        n = byte_array[0] >> 7
        return n
    
    def LSB(self,byte_array):
        """最下位ビットを取得する
        """
        n = self._to_int(byte_array)
        return n & 1

    def anonymize(self,orig_addr):
        """ IPアドレスを匿名化する
        :param orig_addr: IPアドレスの文字列
        :type orig_addr: string
        :return anonymizedIP: int
        :rtype anonymizedIP: int
        """
        # orig_addr を unicode にする
        if not isinstance(orig_addr,unicode):
            orig_addr = unicode(orig_addr)
            
        ip=ipaddress.ip_address(orig_addr)
        addr = int(ip)
        if ip.version == 4:
            N = 32
            ext_addr = addr << 96
        elif ip.version == 6:
            N = 128
            ext_addr = addr 
         
        flip_array = []
        
        for pos in range(N):
            prefix =  ext_addr >> (128-pos) << (128-pos) 
            padded_addr = prefix | (self._padding_int & (2**128-1 >> pos))
            self._input = self.__to_byte_array(padded_addr,16)
            output = array('B')
            output.fromstring(self._cipher.encrypt(self._input))
            flip_array.append(self.LSB(output))
        result = reduce(lambda x, y: (x << 1) | y, flip_array)
        
        anonymizedIP = addr ^ result
        if (ip.version == 4):
            anonymizedIP = ipaddress.IPv4Address(anonymizedIP)
        elif (ip.version  ==6):
            anonymizedIP = ipaddress.IPv6Address(anonymizedIP) 
        return anonymizedIP
    
    def _to_int(self,byte_array):
        """ byte配列をint型に変換する
        """
        return reduce(lambda x,y: x << 8 | y,byte_array) 
    
    def __to_byte_array(self,n,byte_cnt):
        """ int型をbyte配列に変換する
        """
        byte_array = array('B')
        for i in range(byte_cnt):
            byte_array.insert(0, (n >> (i * 8)) & 0xff)
        return byte_array
    
if __name__ == '__main__' :
    # 192.0.2.1 > 2.90.93.17
    # 2001:db8::1 > dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
    key = array('B',range(32))
    obj = CryptoPAn(key)
    ip = '192.0.2.1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.17):",anonymizedIP 
    ip = '192.0.2.2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.19):",anonymizedIP 
    ip = '2001:db8::1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00):",anonymizedIP 
    ip = '2001:db8::2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02):",anonymizedIP   

        

Orig IP: 192.0.2.1
Ano IP(2.90.93.17): 3.202.102.241
Orig IP: 192.0.2.2
Ano IP(2.90.93.19): 3.202.102.243
Orig IP: 2001:db8::1
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00): df63:b23f:ff3c:1c4:7001:e5ff:18c:7e70
Orig IP: 2001:db8::2
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02): df63:b23f:ff3c:1c4:7001:e5ff:18c:7e72


In [37]:
import Crypto.Cipher.AES
import socket
import struct
from array import array
import ipaddress
import numpy as np

#Orig IP: 192.168.0.1
#Ano IP: 2.149.253.242
  

class CryptoPAn:
    def __init__(self,key):
        """ コンストラクタ
        :param key: 暗号化用バイト配列（３２バイト）（上位１６バイトは暗号鍵，下位１６バイトはパディング用）
        :type key: array 
        """
        self._init(key)

    def _init(self,key):
        print "key: ",key
        print "key[:16]:",key[:16]
        print "key[16:]:",key[16:]
        self._cipher = Crypto.Cipher.AES.new(key[:16],Crypto.Cipher.AES.MODE_ECB)
        padding = array('B')
        padding.fromstring(self._cipher.encrypt(key[16:]))
        self._padding_int = self._to_int(padding)
        print "pad: ",padding
        print "_padding_int: ", format(self._padding_int,'0128b')
        
    def changeKey(slef,key):
        """暗号化鍵を変更する
        """
        self._init(key)
    
    def MSB(self,byte_array):
        """最上位ビットを取得する
        """
        n = byte_array[0] >> 7
        return n
    
    def LSB(self,byte_array):
        """最下位ビットを取得する
        """
        n = self._to_int(byte_array)
        return n & 1

    def anonymize(self,orig_addr):
        """ IPアドレスを匿名化する
        :param orig_addr: IPアドレスの文字列
        :type orig_addr: string
        :return anonymizedIP: int
        :rtype anonymizedIP: int
        """
        # orig_addr を unicode にする
        if not isinstance(orig_addr,unicode):
            orig_addr = unicode(orig_addr)
        
        print "orig_addr: ", orig_addr
        ip=ipaddress.ip_address(orig_addr)
        addr = int(ip)
        if ip.version == 4:
            N = 32
            ext_addr = addr << 96
        elif ip.version == 6:
            N = 128
            ext_addr = addr 
         
        flip_array = []
        
        for pos in range(N):
            print "--------------"
            print "pos: ",pos
            prefix =  ext_addr >> (128-pos) << (128-pos) 
            print "prefix:\n",format(prefix,'0128b')
            print "_padding_int:\n", format(self._padding_int,'0128b')
            print "mask:\n",format((2**128-1 >> pos),'0128b')
            print "masked _padding_int:\n",format( (self._padding_int & (2**128-1 >> pos)),'0128b')
            padded_addr = prefix | (self._padding_int & (2**128-1 >> pos))
            print "padded_addr:\n",format(padded_addr,'0128b')
            self._input = self.__to_byte_array(padded_addr,16)
            print "input:",self._input
            print "_input:\n",format(self._to_int(self._input),'0128b')
            output = array('B')
            output.fromstring(self._cipher.encrypt(self._input))
            print "output:",output
            # 論文だとLSBだけど実装はMSB
            flip_array.append(self.MSB(output))
            print "output:\n",format(self._to_int(output),'0128b')
            print "MSB(output):",self.MSB(output)
            print "flip_array: ",flip_array
        print "--------------"
        result = reduce(lambda x, y: (x << 1) | y, flip_array)
        print "addr: \n",format(addr,'0128b')
        print "result: \n",format(result,'0128b')
        
        anonymizedIP = addr ^ result
        print "anonymizedIP: \n",format(anonymizedIP,'0128b')
        if (ip.version == 4):
            anonymizedIP = ipaddress.IPv4Address(anonymizedIP)
        elif (ip.version  ==6):
            anonymizedIP = ipaddress.IPv6Address(anonymizedIP) 
        return anonymizedIP
    
    def _to_int(self,byte_array):
        """ byte配列をint型に変換する
        """
        return reduce(lambda x,y: x << 8 | y,byte_array) 
    
    def __to_byte_array(self,n,byte_cnt):
        """ int型をbyte配列に変換する
        """
        byte_array = array('B')
        for i in range(byte_cnt):
            byte_array.insert(0, (n >> (i * 8)) & 0xff)
        return byte_array
    
if __name__ == '__main__' :
    # 192.0.2.1 > 2.90.93.17
    # 2001:db8::1 > dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
    key = array('B',range(32))
    obj = CryptoPAn(key)
    ip = '192.0.2.1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.17):",anonymizedIP 
    

        

key:  array('B', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
key[:16]: array('B', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
key[16:]: array('B', [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
pad:  array('B', [7, 254, 239, 116, 225, 213, 3, 110, 144, 14, 238, 17, 142, 148, 146, 147])
_padding_int:  00000111111111101110111101110100111000011101010100000011011011101001000000001110111011100001000110001110100101001001001010010011
orig_addr:  192.0.2.1
--------------
pos:  0
prefix:
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
_padding_int:
00000111111111101110111101110100111000011101010100000011011011101001000000001110111011100001000110001110100101001001001010010011
mask:
11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
ma

In [38]:
import Crypto.Cipher.AES
import socket
import struct
from array import array
import ipaddress
import numpy as np

#Orig IP: 192.168.0.1
#Ano IP: 2.149.253.242
  

class CryptoPAn:
    def __init__(self,key):
        """ コンストラクタ
        :param key: 暗号化用バイト配列（３２バイト）（上位１６バイトは暗号鍵，下位１６バイトはパディング用）
        :type key: array 
        """
        self._init(key)

    def _init(self,key):
        self._cipher = Crypto.Cipher.AES.new(key[:16],Crypto.Cipher.AES.MODE_ECB)
        padding = array('B')
        padding.fromstring(self._cipher.encrypt(key[16:]))
        self._padding_int = self._to_int(padding)
        
    def changeKey(slef,key):
        """暗号化鍵を変更する
        """
        self._init(key)
    
    def MSB(self,byte_array):
        """最上位ビットを取得する
        """
        n = byte_array[0] >> 7
        return n
    
    def LSB(self,byte_array):
        """最下位ビットを取得する
        """
        n = self._to_int(byte_array)
        return n & 1

    def anonymize(self,orig_addr):
        """ IPアドレスを匿名化する
        :param orig_addr: IPアドレスの文字列
        :type orig_addr: string
        :return anonymizedIP: int
        :rtype anonymizedIP: int
        """
        # orig_addr を unicode にする
        if not isinstance(orig_addr,unicode):
            orig_addr = unicode(orig_addr)
            
        ip=ipaddress.ip_address(orig_addr)
        addr = int(ip)
        if ip.version == 4:
            N = 32
            ext_addr = addr << 96
        elif ip.version == 6:
            N = 128
            ext_addr = addr 
         
        flip_array = []
        
        for pos in range(N):
            prefix =  ext_addr >> (128-pos) << (128-pos) 
            padded_addr = prefix | (self._padding_int & (2**128-1 >> pos))
            self._input = self.__to_byte_array(padded_addr,16)
            output = array('B')
            output.fromstring(self._cipher.encrypt(self._input))
            # 論文だとLSBだけど実装はMSB
            flip_array.append(self.MSB(output))
        result = reduce(lambda x, y: (x << 1) | y, flip_array)
        
        anonymizedIP = addr ^ (result & (2**128-1 >>8 <<8))
        if (ip.version == 4):
            anonymizedIP = ipaddress.IPv4Address(anonymizedIP)
        elif (ip.version  ==6):
            anonymizedIP = ipaddress.IPv6Address(anonymizedIP) 
        return anonymizedIP
    
    def _to_int(self,byte_array):
        """ byte配列をint型に変換する
        """
        return reduce(lambda x,y: x << 8 | y,byte_array) 
    
    def __to_byte_array(self,n,byte_cnt):
        """ int型をbyte配列に変換する
        """
        byte_array = array('B')
        for i in range(byte_cnt):
            byte_array.insert(0, (n >> (i * 8)) & 0xff)
        return byte_array
    
if __name__ == '__main__' :
    # 192.0.2.1 > 2.90.93.17
    # 2001:db8::1 > dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
    key = array('B',range(32))
    obj = CryptoPAn(key)
    ip = '192.0.2.1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.17):",anonymizedIP 
    ip = '192.0.2.2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.19):",anonymizedIP 
    ip = '2001:db8::1'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00):",anonymizedIP 
    ip = '2001:db8::2'
    anonymizedIP = obj.anonymize(ip)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02):",anonymizedIP   

        

Orig IP: 192.0.2.1
Ano IP(2.90.93.17): 2.90.93.1
Orig IP: 192.0.2.2
Ano IP(2.90.93.19): 2.90.93.2
Orig IP: 2001:db8::1
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e01
Orig IP: 2001:db8::2
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02
